# BACK- [Module 4.1] Deploy from Scratch


여기서는 다음과 같은 작업을 합니다.

- 모델 아티펙트 (model.tar.gz) 파일을 S3에서 로컬에 다운로드
- TF Saved_Model 의 정의를 확인
- SageMaker Model 생성
- Endpoint 생성
- Inference의 Request Serializer and Deserializer 생성
- 프리딕터 생성
- 셈플 데이타로 추론

---
이 노트북은 약 10분 정도 소요 됩니다.


필요한 프로그램 설치

In [1]:
# !pip install -q --upgrade pip
# !pip install -q wrapt --upgrade --ignore-installed
# !pip install -q tensorflow==2.1.0
# !pip install -q transformers==2.8.0
# !pip install -q sagemaker==1.56.1

In [2]:
import boto3
import sagemaker
import pandas as pd
import tensorflow as tf
import os

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

## Download the Model to the Notebook

In [3]:
%store -r training_job_name

In [5]:
training_job_name = 'tensorflow-training-2020-08-02-12-18-28-858'
print(training_job_name)

tensorflow-training-2020-08-02-12-18-28-858


In [6]:
# model_download = 'model'
# os.makedirs(model_download, exist_ok=True)

In [7]:
# !aws s3 cp s3://$bucket/$training_job_name/output/model.tar.gz {model_download}/model.tar.gz

In [8]:
# !tar -xvzf   {model_download}/model.tar.gz
# !saved_model_cli show --all --dir ./tensorflow/saved_model/0/

## SageMaker Model 생성

In [10]:
import os
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://{}/{}/output/model.tar.gz'.format(bucket, training_job_name),
              role=role,
              framework_version='2.0.0') # Elastic Inference does not yet support TF 2.1.0 as of sagemaker==1.56.1

## Endpoint 생성

In [11]:
# instance_type='ml.m4.xlarge'
instance_type = 'local'

deployed_model = model.deploy(initial_instance_count = 1,
                             instance_type = instance_type,
                             wait=True)



Attaching to tmpwyb5y0cc_algo-1-nnley_1
algo-1-nnley_1  | INFO:__main__:starting services
algo-1-nnley_1  | INFO:__main__:using default model name: saved_model
algo-1-nnley_1  | INFO:__main__:tensorflow serving model config: 
algo-1-nnley_1  | model_config_list: {
algo-1-nnley_1  |   config: {
algo-1-nnley_1  |     name: "saved_model",
algo-1-nnley_1  |     base_path: "/opt/ml/model/tensorflow/saved_model",
algo-1-nnley_1  |     model_platform: "tensorflow"
algo-1-nnley_1  |   }
algo-1-nnley_1  | }
algo-1-nnley_1  | 
algo-1-nnley_1  | 
algo-1-nnley_1  | INFO:__main__:nginx config: 
algo-1-nnley_1  | load_module modules/ngx_http_js_module.so;
algo-1-nnley_1  | 
algo-1-nnley_1  | worker_processes auto;
algo-1-nnley_1  | daemon off;
algo-1-nnley_1  | pid /tmp/nginx.pid;
algo-1-nnley_1  | error_log  /dev/stderr error;
algo-1-nnley_1  | 
algo-1-nnley_1  | worker_rlimit_nofile 4096;
algo-1-nnley_1  | 
algo-1-nnley_1  | events {
algo-1-nnley_1  |   worker_connections 2048;
algo-1-nnley_1  | }

In [13]:
endpoint_name = deployed_model.endpoint
print('Endpoint name:  {}'.format(endpoint_name))

Endpoint name:  tensorflow-inference-2020-08-02-12-56-53-567


In [15]:

import json

def input_handler(instances, tokenizer, max_seq_length):
    transformed_instances = []

    for instance in instances:
        encode_plus_tokens = tokenizer.encode_plus(instance,
                                                   pad_to_max_length=True,
                                                   max_length= max_seq_length)

        input_ids = encode_plus_tokens['input_ids']
        input_mask = encode_plus_tokens['attention_mask']
        segment_ids = [0] * max_seq_length

        transformed_instance = {"input_ids": input_ids, 
                                "input_mask": input_mask, 
                                "segment_ids": segment_ids}

        transformed_instances.append(transformed_instance)

    transformed_data = {"instances": transformed_instances}

#    return json.dumps(transformed_data)
    return transformed_data

def output_handler(log_probabilities, classes):
    import tensorflow as tf

#     response_body = response.read().decode('utf-8')

#     response_json = json.loads(response_body)

#     log_probabilities = response_json["predictions"]
    print(log_probabilities)

    predicted_classes = []

    # Convert log_probabilities => softmax (all probabilities add up to 1) => argmax (final prediction)
    for log_probability in log_probabilities:
        softmax = tf.nn.softmax(log_probability)    
        predicted_class_idx = tf.argmax(softmax, axis=-1, output_type=tf.int32)
        predicted_class = classes[predicted_class_idx]
        predicted_classes.append(predicted_class)

    return predicted_classes    

In [19]:
from transformers import DistilBertTokenizer

transformer_tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
max_seq_length = 32
# reviews = ["This is"]
reviews = "['last']"

transformed_input = input_handler(reviews,transformer_tokenizer,max_seq_length )

print(transformed_input)



{'instances': [{'input_ids': [101, 1031, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'segment_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'input_ids': [101, 1005, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'segment_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}, {'input_ids': [101, 1048, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'input_mask': [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'segment_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [20]:
response_json = deployed_model.predict(transformed_input)
log_probabilities = response_json["predictions"]
print("log_probabilities: ", log_probabilities)
classes = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
output_handler(log_probabilities, classes)

log_probabilities: algo-1-nnley_1  | 172.18.0.1 - - [02/Aug/2020:13:05:53 +0000] "POST /invocations HTTP/1.1" 200 1060 "-" "-"
 [[-2.78938413, -2.8333919, -2.30564737, -2.80323434, -0.967011631, -0.674893916, 0.7996611, 6.15671396, -1.28165615, 0.20762749], [-1.46880817, -2.51432586, -3.01440501, -2.2301929, 4.03530264, -2.53345418, 1.70059085, -0.686957419, 0.242017537, -0.176542819], [-2.21749187, -1.99028897, -1.41359711, -2.16797948, -1.1606046, -0.664495, 5.31617498, 2.21805549, -1.97009885, -2.38786364], [-2.16882825, -2.7535584, -2.49117899, -3.0971, -0.506615102, -1.00784576, -0.254950404, 5.20294476, -1.79389775, 2.36805415], [-2.83191109, -2.29273534, -1.39180243, -2.80927444, 2.32096767, -1.76754916, 0.555659115, 4.83452177, -2.2591188, -1.16866124], [-1.41685355, -1.54134703, -1.70415258, -1.31951106, 0.2614429, -1.77130902, 6.72061825, -1.53032529, -0.900431573, -2.29838204], [-1.46880817, -2.51432586, -3.01440501, -2.2301929, 4.03530264, -2.53345418, 1.70059085, -0.686957

[7, 4, 6, 7, 7, 6, 4, 6]

algo-1-nnley_1  | 172.18.0.1 - - [02/Aug/2020:14:24:21 +0000] "GET /ping HTTP/1.1" 200 0 "-" "-"
tmpwyb5y0cc_algo-1-nnley_1 exited with code 137
Aborting on container exit...


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 618, in run
    _stream_output(self.process)
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 677, in _stream_output
    raise RuntimeError("Process exited with code: %s" % exit_code)
RuntimeError: Process exited with code: 137

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sagemaker/local/image.py", line 623, in run
    raise RuntimeError(msg)
RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpwyb5y0cc/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 137



In [ ]:
reviews = "['last']"
deployed_model.predict(reviews)